# CNN Classifier

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout


C:\Users\drake\AppData\Local\Temp\ipykernel_16296\3280155003.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the cleaned and processed dataset
file_path = 'cleaned_movie_data.csv'
cleaned_data = pd.read_csv(file_path)

In [3]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_data['processed_text'], cleaned_data['label'], test_size=0.2, random_state=42)


In [4]:
# Tokenize and pad sequences
max_features = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [5]:
# Build the CNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\drake\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Train the model
batch_size = 64
epochs = 5

history = model.fit(X_train_pad, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=2)


Epoch 1/5
450/450 - 32s - 70ms/step - accuracy: 0.7912 - loss: 0.4245 - val_accuracy: 0.8584 - val_loss: 0.3275
Epoch 2/5
450/450 - 30s - 66ms/step - accuracy: 0.8982 - loss: 0.2560 - val_accuracy: 0.8662 - val_loss: 0.3254
Epoch 3/5
450/450 - 28s - 62ms/step - accuracy: 0.9376 - loss: 0.1667 - val_accuracy: 0.8516 - val_loss: 0.3747
Epoch 4/5
450/450 - 27s - 60ms/step - accuracy: 0.9700 - loss: 0.0885 - val_accuracy: 0.8587 - val_loss: 0.4640
Epoch 5/5
450/450 - 25s - 57ms/step - accuracy: 0.9851 - loss: 0.0435 - val_accuracy: 0.8522 - val_loss: 0.6340


In [7]:
# Evaluate the model
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)

250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step


In [8]:
print("CNN Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

CNN Model Accuracy: 0.857125
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      3966
           1       0.84      0.89      0.86      4034

    accuracy                           0.86      8000
   macro avg       0.86      0.86      0.86      8000
weighted avg       0.86      0.86      0.86      8000



In [9]:
model.save('cnn_model.h5')

In [10]:
import pickle

# Save the model using pickle
with open('cnn_model.pkl', 'wb') as f:
    pickle.dump(model, f)